In [40]:
# from reader import parse_data, list_to_freq_dict, parse_data_test
import xml.etree.ElementTree as ET
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import pprint
import pickle
import os
import json
import numpy as np
import multiprocessing
import time
from numpy import asarray
from numpy import savetxt
pp = pprint.PrettyPrinter(indent=4)

In [2]:
def parse_data(file):
    
    tree = ET.parse(file)
    root = tree.getroot()
    
    data = []
    labels = []
    
    for s_tag in root.iter('s'):
        
        sentence = []
        tags = []
        
        for w_tag in s_tag.iterfind('w'):
            
            word = w_tag.text.replace(" ", "")
            tag = w_tag.attrib['c5']

            sentence.append(word)
            tags.append(tag)
        
        data.append(sentence)
        labels.append(tags)

    return data, labels

In [3]:
def load_dataset(path, training=True):

    if training == True:
        train_data = []
        train_labels = []
        
        for subdir, dirs, files in os.walk(path):
            for file in files:

                fileName = subdir + '/' + str(file)
                data, labels = parse_data(fileName)
                train_data.extend(data)
                train_labels.extend(labels)
            
        return train_data, train_labels
        
    else:
        all_tuples = []
        
        for subdir, dirs, files in os.walk(path):
            for file in files:

                fileName = subdir + '/' + str(file)
                tuples = parse_data_test(fileName)
                all_tuples.extend(tuples)

        return all_tuples

In [4]:
# Load Dataset

train_path = 'Train-corpus/'
test_path = 'Test-corpus/'

data, labels = load_dataset(train_path, training=True)

In [5]:
print(len(data))
print(len(labels))

print(data[0])
print(labels[0])

483629
483629
['Wonder', 'boy', "'s", 'eyes', 'on', 'Wembley']
['VVB-NN1', 'NN1', 'POS', 'NN2', 'PRP', 'NP0']


In [6]:
# Load JSON Files

with open('words.json') as f:
    word_dict = json.load(f)
with open('tags.json') as f:
    tag_dict = json.load(f)
with open('word_tags.json') as f:
    word_tags_dict = json.load(f)

In [7]:
print(len(tag_dict))
print(len(word_dict))
print(len(word_tags_dict))

57
192634
252564


In [170]:
def compute_word_tag_freq_matrix():
    
    rows = len(word_dict.keys())
    cols = len(tag_dict.keys())
    
    mat = [[0 for i in range(cols)] for j in range(rows)] 
    
    i=0
    for word in word_dict.keys():
        j=0
        for tag in tag_dict.keys():
            case = word + "_" + tag
            if case in word_tags_dict.keys():
                mat[i][j] = word_tags_dict[case]
            j = j + 1
        i = i + 1
        
    return mat

In [171]:
freq_matrix = compute_word_tag_freq_matrix()
savetxt('data.csv', freq_matrix, delimiter=',')

In [148]:
def compute_emission_prob_matrix(freq):
    rows = len(freq_matrix)
    cols = len(freq_matrix[0])
    mat = np.array(freq)
    
    for j in range(0, cols):
        total = sum(mat[:, j])
        mat[:, j] = [x/total for x in mat[:, j]]
    
    return mat

In [149]:
emission_mat = compute_emission_prob_matrix(freq_matrix)

In [245]:
def compute_transition_prob_matrix(freq):
    rows = len(freq_matrix)
    cols = len(freq_matrix[0])
    
    mat = np.array(freq, dtype=float)
    
    for i in range(0, rows):
        
        total = float(sum(mat[i]))
        if total == 0:
            total = 1
            
        mat[i] = [x/total for x in mat[i]]
            
    return mat

In [246]:
transition_mat = compute_transition_prob_matrix(freq_matrix)

In [178]:
tag_dict.keys()

dict_keys(['VBB', 'DPS', 'VDG', 'VBG', 'VBD', 'VBI', 'VDZ', 'AV0', 'VBN', 'VDB', 'VHI', 'VDD', 'CRD', 'VHD', 'VDI', 'VBZ', 'DTQ', 'VDN', 'NP0', 'VM0', 'VHB', 'PRF', 'VVZ', 'AJ0', 'VHZ', 'POS', 'VVN', 'PRP', 'VVI', 'VVG', 'VVD', 'VVB', 'XX0', 'EX0', 'CJC', 'AJS', 'VHG', 'ZZ0', 'UNC', 'CJS', 'AJC', 'NN2', 'CJT', 'NN0', 'PNX', 'ITJ', 'PNQ', 'PNP', 'TO0', 'AVP', 'AVQ', 'PNI', 'VHN', 'AT0', 'ORD', 'DT0', 'NN1'])